In [137]:
#from dataclasses import replace
import numpy as np
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from scipy.special import entr
from sklearn.metrics import accuracy_score
import operator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
#from modAL.models import ActiveLearner
# Set our RNG seed for reproducibility.
RANDOM_STATE_SEED = 123
np.random.seed(RANDOM_STATE_SEED)

iris = load_iris()
X_raw = iris['data']
y_raw = iris['target']

In [138]:
# Filter the dataset to include only the first two classes
mask = (y_raw == 1) | (y_raw == 2)
X = X_raw[mask]
y = y_raw[mask]
X_raw=X
y_raw=y
pca = PCA(n_components=2)
transformed_iris = pca.fit_transform(X=X_raw)

# Isolate the data we'll need for plotting.
x_component, y_component = transformed_iris[:, 0], transformed_iris[:, 1]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.6, random_state=42)
n_labeled_examples = X_train.shape[0]
#print(X_train)
#print("OK")

In [139]:
# pick three points randomly and make their labels available (initial training data)
initial_indices=np.random.permutation(n_labeled_examples-1)[0:3]
print(training_indices)

# original training and pool
X_Labelled = X_train[initial_indices]
y_Labelled = y_train[initial_indices]

# Isolate the non-training examples we'll be querying.
X_pool = np.delete(X_train, initial_indices, axis=0)
y_pool = np.delete(y_train, initial_indices, axis=0)

print(X_Labelled.shape)
print(y_Labelled.shape)
print(X_pool.shape)
print(y_pool.shape)

[ 8 69 50]
(3, 4)
(3,)
(37, 4)
(37,)


In [140]:
X_pool_entropy=X_pool
X_pool_dist=X_pool
X_pool_r=X_pool

y_pool_entropy=y_pool
y_pool_dist=y_pool
y_pool_r=y_pool

X_Labelled_entropy=X_Labelled
X_Labelled_dist=X_Labelled
X_Labelled_r=X_Labelled

y_Labelled_entropy=y_Labelled
y_Labelled_dist=y_Labelled
y_Labelled_r=y_Labelled

initial_indices_entropy=initial_indices
initial_indices_dist=initial_indices
initial_indices_r=initial_indices

clf_entropy = RandomForestClassifier(max_depth = 4, min_samples_split=2, n_estimators = 200, random_state = 1) 
clf_dist = RandomForestClassifier(max_depth = 4, min_samples_split=2, n_estimators = 200, random_state = 1) 
clf_r = RandomForestClassifier(max_depth = 4, min_samples_split=2, n_estimators = 200, random_state = 1) 


In [141]:
N_QUERIES = 10
Accuracy_entorpy=[]
Accuracy_dist=[]
Accuracy_random=[]

# Allow our model to query our unlabeled dataset for the most
# informative points according to our query strategy (uncertainty sampling).

for index in range(N_QUERIES):
    clf_entropy.fit(X_Labelled_entropy, y_Labelled_entropy) 
    predictions_entropy= clf_entropy.predict(X_test) 
    Accuracy_entorpy.append(accuracy_score(predictions_entropy, y_test.reshape(-1,1)))
    predicted_probs_entropy = clf_entropy.predict_proba(X_pool_entropy) 
    ## Uncertainty sampling (entropy-based)
    entropy_array = entr(predicted_probs_entropy).sum(axis=1)
    query_index_entropy, value_entropy = max(enumerate(entropy_array), key=operator.itemgetter(1))
    print('The most uncertain point (entropy-based) is No.', query_index_entropy)

    # add the new labeled point to the training data
    initial_indices_entropy =np.append(initial_indices_entropy,query_index_entropy)
    X, y = X_pool_entropy[query_index_entropy].reshape(1, -1), y_pool_entropy[query_index_entropy].reshape(1, )
    
    X_Labelled_entropy=np.concatenate((X_Labelled_entropy,X),axis=0)
    y_Labelled_entropy= np.append(y_Labelled_entropy,y)
    # Remove the queried instance from the unlabeled pool.
    X_pool_entropy, y_pool_entropy = np.delete(X_pool_entropy, query_index_entropy, axis=0), np.delete(y_pool_entropy, query_index_entropy)

    # Distance-based model
    clf_dist.fit(X_Labelled_dist, y_Labelled_dist) 
    predictions_dist= clf_dist.predict(X_test) 
    Accuracy_dist.append(accuracy_score(predictions_dist, y_test.reshape(-1,1)))
    distances = []
    for x_pool in X_pool_dist:
        distances.append(np.mean([np.linalg.norm(x_pool - x_labeled) for x_labeled in X_Labelled_dist]))
    query_index_dist = np.argmax(distances)
    
    print('The distance-based point is No.', query_index_dist)

    # add the new labeled point to the training data
    initial_indices_dist =np.append(initial_indices_dist,query_index_dist)
    X, y = X_pool_dist[query_index_dist].reshape(1, -1), y_pool_dist[query_index_dist].reshape(1, )
    
    X_Labelled_dist=np.concatenate((X_Labelled_dist,X),axis=0)
    y_Labelled_dist= np.append(y_Labelled_dist,y)
    # Remove the queried instance from the unlabeled pool.
    X_pool_dist, y_pool_dist = np.delete(X_pool_dist, query_index_dist, axis=0), np.delete(y_pool_dist, query_index_dist)
    
    # Random method
    clf_r.fit(X_Labelled_r, y_Labelled_r) 
    predictions_r= clf_r.predict(X_test) 
    Accuracy_random.append(accuracy_score(predictions_r, y_test.reshape(-1,1)))
    random_index = np.random.randint(0, len(X_pool_r))
    print('The point randomly is No.', random_index)

    # add the new labeled point to the training data
    initial_indices_r =np.append(initial_indices_r,random_index)
    X, y = X_pool_r[random_index].reshape(1, -1), y_pool_r[random_index].reshape(1, )
    
    X_Labelled_r=np.concatenate((X_Labelled_r,X),axis=0)
    y_Labelled_r= np.append(y_Labelled_r,y)
    # Remove the queried instance from the unlabeled pool.
    X_pool_r, y_pool_r = np.delete(X_pool_r, random_index, axis=0), np.delete(y_pool_r, random_index)


print(Accuracy_entorpy)
print(Accuracy_dist)
print(Accuracy_random)


The most uncertain point (entropy-based) is No. 19
The distance-based point is No. 10
The point randomly is No. 21
The most uncertain point (entropy-based) is No. 31
The distance-based point is No. 5
The point randomly is No. 30
The most uncertain point (entropy-based) is No. 17
The distance-based point is No. 3
The point randomly is No. 27
The most uncertain point (entropy-based) is No. 0
The distance-based point is No. 4
The point randomly is No. 33
The most uncertain point (entropy-based) is No. 22
The distance-based point is No. 0
The point randomly is No. 12
The most uncertain point (entropy-based) is No. 18
The distance-based point is No. 11
The point randomly is No. 8
The most uncertain point (entropy-based) is No. 20
The distance-based point is No. 14
The point randomly is No. 3
The most uncertain point (entropy-based) is No. 16
The distance-based point is No. 2
The point randomly is No. 10
The most uncertain point (entropy-based) is No. 24
The distance-based point is No. 26
Th

In [142]:
import pandas as pd

# Create a pandas DataFrame with the accuracy results
data = {
    'Entropy-based': Accuracy_entorpy,
    'Distance-based': Accuracy_dist,
    'Random': Accuracy_random
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('results_Accuracy.csv', index=False)